In [1]:
# Import Relevant Libraries
import numpy as np
import time

In [2]:
'''
The definitions of Rotation Matrices
--------------------------------------------------------------------------------------------------------------------------------------------------------
    - The rotation abt the 1st principle axis (x-axis) is defined by the angle 'psi'
    - The rotation abt the 2nd principle axis (y-axis) is defined by the angle 'theta'
    - The rotation abt the 3rd principle axis (z-axis) is defined by the angle 'phi'
'''

def rotation_matrix_x(phi):
    """Generate rotation matrix for a roll (rotation about the x-axis)"""
    phi = np.radians(phi)
    c, s = np.cos(phi), np.sin(phi)
    return np.array([[1, 0, 0], 
                     [0, c, s], 
                     [0, -s, c]])

def rotation_matrix_y(theta):
    """Generate rotation matrix for a pitch (rotation about the y-axis)"""
    theta = np.radians(theta)
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, 0, -s], 
                     [0, 1, 0], 
                     [s, 0, c]])

def rotation_matrix_z(psi):
    """Generate rotation matrix for a yaw (rotation about the z-axis)"""
    psi = np.radians(psi)
    c, s = np.cos(psi), np.sin(psi)
    return np.array([[c, s, 0], 
                     [-s, c, 0], 
                     [0, 0, 1]])

In [3]:
def original_apply_rotations(rotation_sequence, target_angles, steps):
    """
    Generates rotation matrices and applies them to an initial frame, based on a specified rotation sequence and corresponding angles.
    """
    # Calculate how many steps each rotation should take
    steps_per_rotation = steps // len(rotation_sequence)
    
    # Map rotation axes to their respective functions
    rotation_functions = {'1': rotation_matrix_x, 
                          '2': rotation_matrix_y, 
                          '3': rotation_matrix_z}

    # Initialize a list to store the rotation matrices for each step
    rotation_matrices = []

    for frame in range(steps + 1):
        # Start with the identity matrix (no rotation)
        R = np.eye(3)
        cumulative_steps = 0

        for index, axis in enumerate(rotation_sequence):
            # Calculate steps taken for the current axis
            axis_steps = min(frame - cumulative_steps, steps_per_rotation)
            
            # Calculate the proportion of the full rotation to apply at the current step
            progress = axis_steps / steps_per_rotation if steps_per_rotation else 0
            angle_deg = progress * target_angles[index]
            
            # Apply the current rotation to the cumulative rotation matrix
            R = np.matmul(rotation_functions[axis](angle_deg), R)
            
            # Update the cumulative steps taken
            cumulative_steps += steps_per_rotation

            # If we've taken all necessary steps for the current frame, break the loop
            if frame < cumulative_steps:
                break
        
        # Store the resulting rotation matrix for the current step
        rotation_matrices.append(R)

    #print(rotation_matrices[2])

    return rotation_matrices



In [22]:
def revised_apply_rotations(rotation_sequence, target_angles, steps):
    """
    Generates rotation matrices and applies them to an initial frame, based on a specified rotation sequence and corresponding angles.
    """
    # Calculate how many steps each rotation should take
    steps_per_rotation = steps // len(rotation_sequence)
    
    # Map rotation axes to their respective functions
    rotation_functions = {'1': rotation_matrix_x, '2': rotation_matrix_y, '3': rotation_matrix_z}

    # Precompute the angles for each step to avoid recalculating them in the loop
    angles = []
    for index, angle in enumerate(target_angles):
        # Create a list of angles for each step of the current rotation
        angle_steps = [(frame / steps_per_rotation) * angle for frame in range(steps_per_rotation + 1)]
        angles.append(angle_steps)

    # Initialize a list to store the rotation matrices for each step
    rotation_matrices = []

    for frame in range(steps + 1):
        # Start with the identity matrix (no rotation)
        R = np.eye(3)
        cumulative_steps = 0

        for index, axis in enumerate(rotation_sequence):
            # If the current frame is before the cumulative steps, break the loop
            if frame < cumulative_steps:
                break

            # Calculate steps taken for the current axis
            axis_steps = min(frame - cumulative_steps, steps_per_rotation)
            if axis_steps > 0:
                # Retrieve the precomputed angle for the current step
                angle_deg = angles[index][axis_steps]
                # Apply the current rotation to the cumulative rotation matrix
                R = np.matmul(rotation_functions[axis](angle_deg), R)

            # Update the cumulative steps taken
            cumulative_steps += steps_per_rotation

        # Store the resulting rotation matrix for the current step
        rotation_matrices.append(R)

    #print(rotation_matrices[2])

    return rotation_matrices

In [23]:
def apply_rotations1(rotation_sequence, target_angles, steps):
    """
    Generates rotation matrices and applies them to an initial frame, based on a specified rotation sequence and corresponding angles.

    Args:
        rotation_sequence (str): The sequence of axes to rotate around, specified as characters (e.g., '321' for rotations around Z, Y, X axes).
        target_angles (tuple): Tuple of target angles in degrees, each element corresponding to the rotation about the axis in the rotation_sequence.
        steps (int): Total number of steps in the animation, dictating the granularity of the rotation application.

    Returns:
        list: A list of rotation matrices corresponding to each step in the animation, representing the frame's orientation at each step.
    """
    rotation_functions = {'1': rotation_matrix_x, 
                          '2': rotation_matrix_y, 
                          '3': rotation_matrix_z}

    steps_per_rotation = steps // len(rotation_sequence)
    angles = [[(frame / steps_per_rotation) * angle for frame in range(steps_per_rotation + 1)] 
              for angle in target_angles]
    
    rotation_matrices = []

    for frame in range(steps + 1):
        R = np.eye(3)
        cumulative_steps = 0

        for index, axis in enumerate(rotation_sequence):
            if frame < cumulative_steps:
                break

            axis_steps = min(frame - cumulative_steps, steps_per_rotation)
            if axis_steps > 0:
                angle_deg = angles[index][axis_steps]
                R = np.matmul(rotation_functions[axis](angle_deg), R)

            cumulative_steps += steps_per_rotation

        rotation_matrices.append(R)

    #print(rotation_matrices[2])

    return rotation_matrices

In [24]:
# Benchmark
rotation_sequence = '213'
target_angles = (45, 45, 45)
steps = 30

start_time = time.time()
original_apply_rotations(rotation_sequence, target_angles, steps)
original_time = time.time() - start_time

start_time = time.time()
revised_apply_rotations(rotation_sequence, target_angles, steps)
revised_time = time.time() - start_time

start_time = time.time()
apply_rotations1(rotation_sequence, target_angles, steps)
revised_time1 = time.time() - start_time

print(f"Original Time: {original_time:.6f} seconds")
print(f"Revised Time : {revised_time:.6f} seconds")
print(f"Revised Time1 : {revised_time1:.6f} seconds")

11
Original Time: 0.001086 seconds
Revised Time : 0.000000 seconds
Revised Time1 : 0.000000 seconds
